In [1]:
import os
os.chdir("../")
%pwd

'/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars'

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Dropout, MaxPooling2D, Conv2DTranspose

2023-09-25 22:19:57.077781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 22:19:57.698061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareModelConfig:
    img_height: int
    img_width :int
    num_channels: int
    input_size: tuple
    n_classes: int

In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
from box import ConfigBox
from src import logger
from src.utils.buildingblocks import conv_block, upsampling_block

In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) :
                 
        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        self.config= ConfigBox(self.config[0]) # configbox is returning tuble so to avoid it we are again applying configbox to the dictionary at index 0
        self.params= ConfigBox(self.params[0])



    def get_prepare_model_config(self)-> PrepareModelConfig:
        config = self.params



        prepare_model_config = PrepareModelConfig(
                            img_height = config.img_height,
                            img_width = config.img_width,
                            num_channels= config.num_channels,
                            input_size = config.input_size,
                            n_classes= config.n_classes)
        return prepare_model_config

In [5]:
img_height = 192
img_width = 256
num_channels = 3

## Encoder Block

In [6]:
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):

    layer = Conv2D(n_filters, kernel_size= 3, padding='same', kernel_initializer='he_normal')(inputs)
    layer = BatchNormalization(axis=3)(layer,training=True)
    layer = LeakyReLU()(layer)
    layer = Conv2D(n_filters, 3, padding='same', kernel_initializer='he_normal')(layer)
    layer = BatchNormalization(axis=3)(layer,training= False)
    layer = LeakyReLU()(layer)
    
    if dropout_prob > 0:
        layer = Dropout(dropout_prob)(layer)
        
    if max_pooling:
        next_layer = MaxPooling2D(pool_size=(2,2))(layer)
        
    else:
        next_layer = layer
        
    skip_connection = layer
    
    return next_layer, skip_connection

## Decoder Block

In [7]:
def upsampling_block(expansive_input, contractive_input,n_filters=32):

    up = Conv2DTranspose(n_filters, kernel_size = 3, strides=(2,2), padding='same')(expansive_input)
    merge = tf.concat([up,contractive_input],axis =3)


    layer = Conv2D(n_filters, kernel_size= 3, activation='relu', padding='same',
                 kernel_initializer= 'he_normal')(merge)
    layer = BatchNormalization(axis=3)(layer, training= False)
    layer = LeakyReLU() (layer)

    layer = Conv2D(n_filters, kernel_size= 3, activation='relu', padding='same',
                 kernel_initializer= 'he_normal')(layer)
    layer = BatchNormalization(axis=3)(layer, training= False)
    layer = LeakyReLU() (layer)

    return layer

## Building the model

In [12]:
def unet_model(input_size=(96,128,3), n_filters=32, n_classes = 13):

    # Encoder
    inputs = tf.keras.Input(input_size)

    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], n_filters* 2)
    cblock3 = conv_block(cblock2[0], n_filters * 4)
    cblock4 = conv_block(cblock3[0], n_filters * 8, dropout_prob=0.3)

    cblock5 = conv_block(cblock4[0], n_filters * 16, dropout_prob=0.3, max_pooling=False)

    # Decoder

    ublock6 = upsampling_block(cblock5[0], cblock4[1], n_filters *8)
    ublock7 = upsampling_block(ublock6, cblock3[1], n_filters *4)
    ublock8 = upsampling_block(ublock7, cblock2[1], n_filters * 2)
    ublock9 = upsampling_block(ublock8, cblock1[1], n_filters)

    layer = Conv2D(n_filters, 3, padding='same', kernel_initializer='he_normal')(ublock9)
    layer = BatchNormalization(axis =3)(layer, training=False)
    layer = LeakyReLU()(layer)

    layer = Conv2D(n_classes, 1, padding='same')(layer)
    layer = BatchNormalization(axis =3)(layer, training=False)
    layer = LeakyReLU()(layer)

    model = tf.keras.Model(inputs = inputs, outputs= layer)
    return model

In [ ]:
class PrepareModel:
    def __init__(self, config: PrepareModelConfig):
        self.input_size = config.input_size
        self.n_classes = config.n_classes


    def unet_model(self, n_filters= 32):
        input_size = self.input_size
        print(type(input_size))
        n_classes = self.n_classes

        # Encoder
        
        inputs = tf.keras.Input(input_size)
    
        cblock1 = conv_block(inputs, n_filters)
        cblock2 = conv_block(cblock1[0], n_filters* 2)
        cblock3 = conv_block(cblock2[0], n_filters * 4)
        cblock4 = conv_block(cblock3[0], n_filters * 8, dropout_prob=0.3)
        cblock5 = conv_block(cblock4[0], n_filters * 16, dropout_prob=0.3, max_pooling=False)
    

        # Decoder
    
        ublock6 = upsampling_block(cblock5[0], cblock4[1], n_filters *8)
        ublock7 = upsampling_block(ublock6, cblock3[1], n_filters *4)
        ublock8 = upsampling_block(ublock7, cblock2[1], n_filters * 2)
        ublock9 = upsampling_block(ublock8, cblock1[1], n_filters)
    
        layer = Conv2D(n_filters, 3, padding='same', kernel_initializer='he_normal')(ublock9)
        layer = BatchNormalization(axis =3)(layer, training=False)
        layer = LeakyReLU()(layer)
    
        layer = Conv2D(n_classes, 1, padding='same')(layer)
        layer = BatchNormalization(axis =3)(layer, training=False)
        layer = LeakyReLU()(layer)
    
        model = tf.keras.Model(inputs = inputs, outputs= layer)
        
        return model

In [13]:


unet = unet_model()

In [14]:
unet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 96, 128, 3)]         0         []                            
                                                                                                  
 conv2d_20 (Conv2D)          (None, 96, 128, 32)          896       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_20 (Ba  (None, 96, 128, 32)          128       ['conv2d_20[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 leaky_re_lu_20 (LeakyReLU)  (None, 96, 128, 32)          0         ['batch_normalization_20

In [15]:
history = unet.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
try:

    config = ConfigurationManager()
    prepare_model_config = config.get_prepare_model_config()
    prepare_model = PrepareModel(config = prepare_model_config)
    unet = prepare_model.unet_model()
    unet.summary()

except Exception as e:
    logger.info(e)
    raise e

[2023-09-24 21:24:17,191: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-24 21:24:17,194: INFO: common: yaml fileparams.yamlloaded Successfully]
<class 'box.box_list.BoxList'>


2023-09-24 21:24:17.206826: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-24 21:24:17.257020: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-24 21:24:17.257209: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 96, 128, 3)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 96, 128, 32)          896       ['input_5[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 96, 128, 32)          128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 96, 128, 32)          0         ['batch_normalization[0][0

In [36]:
l = '(2,3,4)'


('(', '2', ',', '3', ',', '4', ')')